In [19]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [10]:
[ int(x) for x in ReportCycle.objects.all()[0].reportCycle.split(',')]

[3, 10, 27, 37, 57]

In [60]:
 def update_initial_data(self):
        phone_list = [ p.phone_no for p in self.phone_set.all()]
        qs = Phone.objects.filter(measdate=self.measdate, phone_no__in=phone_list).exclude(phoneGroup=self)
        if qs.exists():
            measuringTeam = None
            for p in qs:
                print(p, p.phoneGroup.id, p.phoneGroup.measuringTeam)
                if p.phoneGroup.measuringTeam and p.phoneGroup.measuringTeam != None:
                    measuringTeam = p.phoneGroup.measuringTeam
                    break
        print(measuringTeam)

In [61]:
pg = PhoneGroup.objects.get(pk=4)

In [62]:
update_initial_data(pg)

경상남도-사천시-남양동/행-용택-1/1029213866/41 3 1조
1조


In [65]:
phone = Phone.objects.get(id=1)

In [67]:
        # 측정 단말기의 DL/UP 평균값들을 가져온다.
        dl_sum, ul_sum, dl_count, ul_count = 0, 0, 0, 0
        dl_nr_count, ul_nr_count = 0, 0 # 5G->LTE 전환콜수(DL, UL)ß
        avg_downloadBandwidth = 0  # 다운로드 평균속도
        avg_uploadBandwidth = 0  # 업로드 평균속도
        # nr_count = 0 # 5G->NR 전환 콜수
   

        # 2022.02.26 - 데이터가 맞지 않아 재작성 함
        #            - 속도평균값을 산출할 때 고민해야 하는 사항은 몇번째 턴인지, 현재 콜카운트, 총 측정횟수 등을 고려해야 한다.
        #                       <--- tern 1 ----> <--- tern 2 --->
        #            - Phone 1: 1,2,3,4,5,6,...27,1,2,3,4,5,...
        #              Phone 2: 1,  3,4,5,6,...27,1,2,3,  5,... * 콜이 누락되는 경우도 있음
        #            - tern : 현재 콜카운트가 증가하다 감소하면 한개의 턴(측정주기, tern)이 끝난 것으로 판단
        #                     즉, 현재 콜카운트가 3이라면 첫번째 측정주기의 3번째 콜인지? 두번째 측정주기의 3번째 콜인지? 확인해서
        #                     상대편 측정단말의 속도평균을 산출할때 동일한 기준을 적용해야 한다.   
        #
        # 2022.03.10 currentCount -> phone.total_count로 변경 적용
        # 메시지를 보내려고 하는 측정 단말기
        total_count = 0
        for m in phone.measurecalldata_set.filter(testNetworkType='speed').order_by("meastime"):
            if total_count >= phone.total_count: break
            if m.phone.networkId == '5G' and m.networkId == 'NR':
                # 측정 단말이 5G이고, 측정 데이터가 NR이면 5G->NR 전환 콜수를 하나 증가시킨다. 
                if m.downloadBandwidth and m.downloadBandwidth > 0:
                    dl_nr_count += 1
                if m.uploadBandwidth and m.uploadBandwidth > 0:
                    ul_nr_count += 1
            else: 
                # 속도 평균값을 구하기 위한 속도 합계와 콜 카운트를 누적한다. 
                if m.downloadBandwidth and m.downloadBandwidth > 0:
                    dl_sum +=  m.downloadBandwidth
                    dl_count += 1
                if m.uploadBandwidth and m.uploadBandwidth > 0:
                        ul_sum += m.uploadBandwidth
                        ul_count += 1
            # 5G->LTE 전환포함하여 콜 카운트를 산정한다.
            total_count += 1
            # print(f"###-1 {phone.phone_no}/{m.currentCount}/{m.downloadBandwidth}/{m.uploadBandwidth }///", mdata.currentCount)
           
        # 상대편 측정 단말기
        total_count = 0
        qs = phone.phoneGroup.phone_set.filter(ispId='45008', manage=True).exclude(phone_no=phone.phone_no)
        if qs.exists():
            oPhone = qs[0]
            for m in oPhone.measurecalldata_set.filter(testNetworkType='speed').order_by("meastime"):
                # 2022.02.25 DL/UL 측정건수가 10건 이상 차이가 나지 않는다는 가정에서 아래 코드가 정상 동작한다.
                if total_count >= phone.total_count: break
                if m.phone.networkId == '5G' and m.networkId == 'NR':
                    # 측정 단말이 5G이고, 측정 데이터가 NR이면 5G->NR 전환 콜수를 하나 증가시킨다. 
                    if m.downloadBandwidth and m.downloadBandwidth > 0:
                        dl_nr_count += 1
                    if m.uploadBandwidth and m.uploadBandwidth > 0:
                        ul_nr_count += 1
                else: 
                    # 속도 평균값을 구하기 위한 속도 합계와 콜 카운트를 누적한다. 
                    if m.downloadBandwidth and m.downloadBandwidth > 0:
                        dl_sum +=  m.downloadBandwidth
                        dl_count += 1
                    if m.uploadBandwidth and m.uploadBandwidth > 0:
                            ul_sum += m.uploadBandwidth
                            ul_count += 1
                # 5G->LTE 전환포함하여 콜 카운트를 산정한다.
                total_count += 1
                # print(f"###-2 {oPhone.phone_no}/{m.currentCount}/{m.downloadBandwidth}/{m.uploadBandwidth }///", mdata.currentCount)

        # DL/UL 평균속도를 산출한다.         
        if dl_count > 0 : avg_downloadBandwidth = round(dl_sum / dl_count,2)
        if ul_count > 0 : avg_uploadBandwidth = round(ul_sum / ul_count,2)
        # if nr_count > 0 : avg_nrRate = round(nr_count / (dl_count + ul_count) * 100,2)
        
        print(dl_nr_count, ul_nr_count)

0 1


In [7]:
self = PhoneGroup.objects.get(id=8)

In [8]:
    def update_initial_data(self):
        phone_list = [ p.phone_no for p in self.phone_set.all()]
        qs = Phone.objects.filter(measdate=self.measdate, phone_no__in=phone_list).exclude(phoneGroup=self)
        print(qs)
        if qs.exists():
            measuringTeam = None
            for p in qs:
                print(p, p.phoneGroup.id, p.phoneGroup.measuringTeam)
                if p.phoneGroup.measuringTeam and p.phoneGroup.measuringTeam != None:
                    measuringTeam = p.phoneGroup.measuringTeam
                    break
            self.measuringTeam = measuringTeam
            self.save()

In [9]:
update_initial_data(self)

<QuerySet [<Phone: 동내면/커버리지 1조/1072010298/1>]>
동내면/커버리지 1조/1072010298/1 1 1조


In [27]:
mdata = MeasureCallData.objects.get(id=1)
# phone_no_list = mdata.phone.phoneGroup.phone_set.all().values_list('phone_no', flat=True)
phone_list = mdata.phone.phoneGroup.phone_set.all()
print(phone_list)
gs = Message.objects.filter(phone__in=phone_list, phone__status='START_M')
gs

<QuerySet [<Phone: 서울특별시-금천구-가산동/행-조영-1/1029213855/1>, <Phone: 서울특별시-금천구-가산동/행-조영-1/1029213942/1>]>


<QuerySet [<Message: Message object (1)>]>

In [18]:
mdata = MeasureCallData.objects.get(id=163)
mdata.phone.phoneGroup
qs = Phone.objects.filter(phoneGroup=mdata.phone.phoneGroup, meastype='DL')
qs

<QuerySet []>

In [ ]:
mdata = MessureCallData.objects.filter(meastime=, phone_no=)

In [20]:
def current_count_check(mdata):
    """DL/UL 측정단말의 현재 콜카운트와 보고기준 콜카운트를 확인한다."""
    result = False
    phone = mdata.phone
    # 해당지역에 단말이 첫번째로 측정을 시작했는지 확인한다.
    # print("current_count_check()-total_count", phone.total_count)
    if mdata.currentCount == 1:
        # # 해당일자에 첫번째 측정 단말기일 경우, 측정시작 메시지를 전송한다. 
        # # 즉, 해당일자에 측정중인 단말이 없다면 메시지를 전송한다.
        # qs = Phone.objects.filter(measdate=phone.measdate, manage=True).exclude(phone_no=phone.phone_no)
        # if not qs.exists():
        #     result = True
        # 측정시작 조건 : 현재 콜카운트가 1인 다른 측정 데이터가 있는지 확인
        # - 결과 <= 1건 : 자기 자신밖에 없으니 측정시작 메시지 전송
        # - 결과 > 1건 : 이미 측정시작 메시지를 전송했으니 메시지를 전송하지 않음
        # 2022.03.11 - 1)전체대상 측정시작 메시지(START_F)
        meastime_from = int(str(mdata.meastime)[:8] + '000000000') # 조회시작
        meastime_to = int(str(mdata.meastime)[:8] + '235959999') # 조회종료
        qs = MeasureCallData.objects.filter(Q(meastime__gte=meastime_from) & Q(meastime__lte=meastime_to))
        if mdata.phone.status == 'START_F' and len(qs) <= 1:
            result = True
        
        # 2022.03.11 - 2)해당지역 측정시작 메시지(START_M)
        # 2-1) 상대편 측정 단말기가 등록되어 있는지 확인한다. 
        # 2-2) 상대편 측정 단말기에 속도 측정 데이터가 있는지 확인한다.
        # 2022.03.15 - 측정 데이터가 첫번째 데이터인지 확인하는 사이에 상대편 단말기에 의한 측정 데이터가 생성되어 해당 지역측정 시작 메시지가 
        #              누락되는 현상을 막기 위해 명확하게 전송 메시지 내에 단말기 상태를 가져감
        #            - 단말그룹으로 묶여 있는 측정 단말기들로 측정시작 메시지가 전송되었는지를 확인하여 측정시작 메시지를 전송하게 함 
        elif mdata.phone.status == 'START_M' and mdata.phone.manage == True:
        #     qs = mdata.phone.phoneGroup.phone_set.exclude(phone_no=mdata.phone_no)
        #     if qs.exists():
        #         oPhone = qs[0]
        #         qs = oPhone.measurecalldata_set.filter(currentCount=1, testNetworkType='speed')
        #         if not qs.exists():
        #             result = True
            phone_list = mdata.phone.phoneGroup.phone_set.all()
            qs = Message.objects.filter(phone__in=phone_list, status='START_M')
            if not qs.exists():
                result = True 

    # elif mdata.currentCount in [3, 10, 27, 37, 57,]:
    # 2022.03.10 currentCount -> phone.total_count로 변경 적용
    elif phone.total_count in [ int(x) for x in ReportCycle.objects.all()[0].reportCycle.split(',')]:
        # 단말기 체인지 되고 재측정시 그데이터도 더해져서 메시지가 보내질수도 있다 그때는 예외조건
        # 단밀기 그룹으로 묶여 았는 상대편 측정 단말기를 조회한다.
        qs = phone.phoneGroup.phone_set.exclude(phone_no=phone.phone_no)
        if qs.exists():
            p = qs[0]
            # 상대편 측정 단말기의 현재 콜 카운트가 측정 단말 보다 같거나 커야 한다.
            if p.total_count >= phone.total_count:
                result = True
        # 2022.02.26 - 측정단말이 하나인 경우 어떻게 처리해야 할지 고민이 필요하다.
        # 2022.03.05 - 속도측정의 경우 대부분 2대의 측정 단말기를 가지고 진행을 하며, 최소한 3콜 이전에는 2대 모두의 측정 데이터가
        #              발생한다는 가정으로 진행함
        #              즉, 3콜에도 상대편 단말기의 측정 데이터가 없으면 1개의 측정 단말기로 측정을 진행한다고 생각하고,
        #              보고주기에 따라 메시지를 보냄
        # Case 1 : 1,2,3,4,5,...27,1,2,3,4,5,..27,.. (DL을 측정하고, 이후 UL을 측정하는 경우)
        # Case 1 : 1,1,2,2,3,3,4,4,5,5,6,6,...27,27,.. (DL과 UL을 번갈아 한번씩 측정하는 경우)
        else:
            pass
            # 2022.03.10 currentCount -> phone.total_count로 변경 적용하면서 추가 고민해야 함
            # qs = MeasureCallData.objects.filter(phone_no=mdata.phone_no, currentCount__lte=mdata.currentCount)
            # if qs.exists() and qs.count() > mdata.currentCount:
            #    qs = qs.filter(currentCount = mdata.currentCount)
            #    if qs.count() > 1:
            #         result = True
            # else:
            #     result = True

    return result